Signal- und Systemtheorie / Signals and Systems \#24015

Dr. Frank Schultz, Prof. Sascha Spors

Institut für Nachrichtentechnik (INT),
Fakultät für Informatik und Elektrotechnik (IEF),
Universität Rostock

Institute of Communications Engineering,
Faculty of Computer Science and Electrical Engineering,
University of Rostock

**Übung / Exercise 9**, Sommer / Summer 2020

[Signals- & Systems](https://github.com/spatialaudio/signals-and-systems-exercises),
[University of Rostock](https://www.uni-rostock.de/en/),
[Institute of Communications Engineering](https://www.int.uni-rostock.de/),
Prof. [Sascha Spors](https://orcid.org/0000-0001-7225-9992),
[Frank Schultz](https://orcid.org/0000-0002-3010-0294),
Till Rettberg,
[CC BY 4.0](https://creativecommons.org/licenses/by/4.0/)

In [ ]:
from scipy.signal import tf2zpk, bilinear
import numpy as np
import matplotlib.pyplot as plt
import sys
import os
cur_fol = os.getcwd()
print(cur_fol)
sys.path.append(cur_fol + '/../')

from sig_sys_tools import plot_dtlti_analysis

# Analysis of Discrete-Time, Linear, Time-Invariant System (DTLTI)

Evaluate and plot the

* impulse response
* step response
* frequency response (level, phase, group delay)
* zero/pole/gain mapping, i.e. z-plane plot

of a DTLTI system given as z-transfer function

\begin{equation}
H(z) = \frac{\sum\limits_{n=0}^{N} b_n z^{-n}}{\sum\limits_{m=0}^{M} a_m z^{-m}}=\frac{b_0 + b_1 z^{-1} + b_2 z^{-2} + ...}{a_0 + a_1 z^{-1} + a_2 z^{-2} + ...}
\end{equation}

using `scipy.signal` routines for discrete-time domain signal processing. The coefficients $b_0, b_1,b_2,...,b_N$ are stored in vector/array b, the coefficients $a_0, a_1,a_2,...,a_M$ are stored in vector/array a.

Most often $H(z)$ is normalized such that $a_0=1$.

Note that `signal.dlti` handling has a known issue (https://dsp.stackexchange.com/questions/40312/why-does-pythons-scipy-signal-dimpulse-introduce-delay-in-impulse-response).
For workaround, **we must ensure** that **a and b** have the **same length** by suitable zero padding.
This is tedious for a long FIR.
In the context of this notebook for didactical purpose, this handling might be acceptable.
If analysis of long FIRs is needed, we might want to implement a optimized DFT-based handling on our own, until this issue might be fixed in `scipy.signal`.

Used abbreviations:

- DFT...discrete Fourier transform

- DTFT...discrete-time Fourier transform

- FIR...finite impulse response, i.e. we actually refer to a non-recursive system

- IIR...infinite impulse response, i.e. we actually refer to a recursive system

- DTLTI...discrete-time, linear, time-invariant

- ROC...region of convergence for z-transform.

Since we aim at causal impulse responses, ROC must be $|z|>\mathrm{max}(|z_{\infty}|)$.
Thus, if ROC (white) includes the unit circle (along which DTFT is defined), this implies that all poles are within the unit circle and by that that the causal system response is stable.

## Evaluating and Plotting Routine

The plotting routine `plot_dtlti_analysis(z, p, k, fs=1, Nf=2**10, Nt=2**5)` is found in the `sig_sys_tools.py`.

We demonstrate frequency axis handling with

* logarithmic x-axis along f / Hz for level, top, red
* linear x-axis along $\frac{\Omega}{2\pi} = \frac{f}{f_s}$ for level, bottom, blue
* linear x-axis along $\Omega$ for phase
* linear x-axis along $\frac{\Omega}{\pi}$ for group delay

in the subplots below.
We should get familiar with these different styles, since they often occur and have pros / cons in different applications / visualization strategies.
The choice of which response maps to which handling is rather arbitrary and we can freely switch to whatever style is best suitable.

We discuss systems that were analytically evaluated in the tutorials. Feel free to play around with other systems that you find in textbooks, online material or that come up by your own.

## Pass Thru = Digital Wire

\begin{equation}
H(z) = 1
\end{equation}


In [ ]:
# pass through
b = [+1]
a = [+1]
z, p, k = tf2zpk(b, a)
plot_dtlti_analysis(z, p, k)
plt.savefig('PassThru.pdf')

## Exercise 7.3: System H1

\begin{align}
H(z) = \frac{z^3 +z^2 -z + \frac{1}{2}}{z^3} = 1 + z^{-1} - z^{-2} + \frac{1}{2} z^{-3}
\end{align}

In [ ]:
# FIR filter, finite impulse response vector h would be handled as
b = [1, 1, -1, 1/2]  # = h
a = [1, 0, 0, 0]  # len(a)==len(b) handling :-(
z, p, k = tf2zpk(b, a)
plot_dtlti_analysis(z, p, k)
plt.savefig('System_UE7_3_H1.pdf')

## Exercise 7.3: System H3

\begin{align}
H(z) = \frac{2 z^2 + 1}{z^2-\frac{1}{2} z} = \frac{2 + z^{-2}}{1-\frac{1}{2} z^{-1}}
\end{align}


In [ ]:
# IIR filter
b = [2, 0, 1]
a = [1, -1/2, 0]  # note the sign reversal of coef a1 compared to block diagram
z, p, k = tf2zpk(b, a)
plot_dtlti_analysis(z, p, k)
plt.savefig('System_UE7_3_H3.pdf')

## Exercise 94A7A6D9E9

\begin{equation}
H(z) = \frac{z^2-z+2}{z^2-\frac{1}{2} z + \frac{1}{4}}=\frac{1-z^{-1}+2 z^{-2}}{1-\frac{1}{2} z^{-1} + \frac{1}{4} z^{-2}}
\end{equation}


In [ ]:
# IIR filter
b = [+1, -1, +2]
# note the sign reversal of coefs a1, a2 compared to block diagram
a = [+1, -1/2, +1/4]
z, p, k = tf2zpk(b, a)
plot_dtlti_analysis(z, p, k)
plt.savefig('system_94A7A6D9E9.pdf')

## Discrete-Time Version of a Laplace-Domain Transfer Function

The analog lowpass filter of 2nd order from

https://github.com/spatialaudio/signals-and-systems-exercises/blob/master/laplace_system_analysis/lowpass2nd_44EB4169E9.ipynb

\begin{equation}
H(s) = \frac{1}{\frac{16}{25} s^2 + \frac{24}{25} s + 1}
\end{equation}

is transfered to discrete-time domain by setting the sampling frequency 100 times above the cutoff frequency and using the so called bilinear transform. This design exhibits a zero on the unit circle at $z=-1$ for lowpass characteristics.

The 2nd order differential equation leads to a 2nd order difference equation, which can be interpreted as a 2nd order recursive filter, usually referred to as infinite impulse response (IIR) filter.

In [ ]:
# digital filter design with so called bilinear transform of a
# continuous-time ODE (we will learn this in detail in the DSP course):
# ODE RLC-example from 'solving_2nd_order_ode.pdf' /
# 'frequency_response_2nd_order_ode.pdf' is
# 16/25 y''(t) + 24/25 y'(t) + y(t) = DiracDelta(t), y'(t-=0)=0, y(t-=0)=0
# for example sampled with
# sampling frequency in Hz, note: omega0 = 5/4
fs = np.ceil((5/4)/(2*np.pi)*100)
# note that we just round up to integer for nicer plotting
print('fs = ', fs, 'Hz')
[b, a] = bilinear([25/16], [1, 24/16, 25/16], fs=fs)
print('b = ', b)
print('a = ', a)

z, p, k = tf2zpk(b, a)
plot_dtlti_analysis(z, p, k, fs=fs, Nt=np.int(np.ceil(fs*10)))

# we obtain a discrete-time 2nd order IIR-filter with lowpass characteristics
plt.savefig('AnalogODE_Bilinear.pdf')

# Copyright

This tutorial is provided as Open Educational Resource (OER), to be found at
https://github.com/spatialaudio/signals-and-systems-exercises
accompanying the OER lecture
https://github.com/spatialaudio/signals-and-systems-lecture.
Both are licensed under a) the Creative Commons Attribution 4.0 International
License for text and graphics and b) the MIT License for source code.
Please attribute material from the tutorial as *Frank Schultz,
Continuous- and Discrete-Time Signals and Systems - A Tutorial Featuring
Computational Examples, University of Rostock with
``main file, github URL, commit number and/or version tag, year``.